## [0단계] 사용할 라이브러리와 패키지 불러오기

In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## [1단계] 예측 모델 선정

In [ ]:
df = pd.read_csv("../data/merged/traffic_weather_with_cluster.csv")     # (클러스터 포함) 최종 데이터 로드
df['datetime'] = pd.to_datetime(df['datetime'])                         # datetime 형 변환

df['date'] = df['datetime'].dt.floor('D')                               # 날짜 추출 
df['hour'] = df['datetime'].dt.hour                                     # 시간 추출
df["is_rain"] = (df["precipitation"] > 0).astype(int)                   # 강수 여부 이진 변수 생성 

df = df.sort_values(["intersection", "datetime"])                       # 교차로별 시간순 정렬 

df["lag1"] = df.groupby("intersection")["traffic_volume"].shift(1)      # 1시간 전 교통량
df["lag2"] = df.groupby("intersection")["traffic_volume"].shift(2)      # 2시간 전 교통량    
df["lag3"] = df.groupby("intersection")["traffic_volume"].shift(3)      # 3시간 전 교통량

df["roll3"] = (   # 3시간 이동 평균
    df.groupby("intersection")["traffic_volume"]
    .rolling(3).mean()
    .reset_index(0,drop=True)
)   
df["roll6"] = (   # 6시간 이동 평균  
    df.groupby("intersection")["traffic_volume"]
    .rolling(6).mean()
    .reset_index(0,drop=True)
)   

df = df.dropna()    # lag, rolling으로 생긴 결측치 제거

# Train / Test 분리 (최근 30일)
test_size_days = 30
split_date = df["date"].max() - pd.Timedelta(days=test_size_days)   # 분기 기준 날짜 설정

train = df[df["date"] <= split_date]    # 학습 데이터
test  = df[df["date"] > split_date]     # 테스트 데이터

drop_cols = ["traffic_volume", "intersection", "datetime", "date"]

X_train = train.drop(columns=drop_cols)    # 학습 입력 변수 
y_train = train["traffic_volume"]          # 학습 타겟 변수 

X_test = test.drop(columns=drop_cols)      # 테스트 입력 변수
y_test = test["traffic_volume"]            # 테스트 타겟 변수 

# 후보 1) Linear Regression

lr = LinearRegression()     # 선형 회귀 모델 생성
lr.fit(X_train, y_train)    # 모델 학습

pred_lr = lr.predict(X_test)    # 테스트 예측

rmse_lr = mean_squared_error(y_test, pred_lr) ** 0.5    # 선형 회귀의 RMSE
mae_lr = mean_absolute_error(y_test, pred_lr)           # 선형 회귀의 MAE

print("Baseline (LR) RMSE:", rmse_lr)
print("Baseline (LR) MAE:", mae_lr)

# 후보 2) RandomForest
rf = RandomForestRegressor(     # 랜덤 포레스트 모델 생성
    n_estimators=200,           # 트리 개수 
    max_depth=12,               # 트리 최대 깊이
    random_state=42
)

rf.fit(X_train, y_train)        # 모델 학습
pred_rf = rf.predict(X_test)    # 테스트 예측

rmse_rf = mean_squared_error(y_test, pred_rf) ** 0.5    # 랜덤 포레스트의 RMSE
mae_rf = mean_absolute_error(y_test, pred_rf)           # 랜덤 포레스트의 MAE

print("RF RMSE:", rmse_rf)
print("RF MAE:", mae_rf)

# 후보 3) XGBoost
xgb = XGBRegressor(         # XGBoost 모델 생성
    n_estimators=500,       # 부스팅 트리 개수
    max_depth=8,            # 트리 최대 깊이
    learning_rate=0.05,     # 학습률
    subsample=0.8,          # 샘플 비율
    colsample_bytree=0.8,   # 특성 샘플링 비율
    random_state=42
)

xgb.fit(X_train, y_train)       # 모델 학습
pred_xgb = xgb.predict(X_test)  # 테스트 예측

rmse_xgb = mean_squared_error(y_test, pred_xgb) ** 0.5      # XGBoost의 RMSE   
mae_xgb = mean_absolute_error(y_test, pred_xgb)             # XGBoost의 MAE

print("XGB RMSE:", rmse_xgb)
print("XGB MAE:", mae_xgb)


Baseline (LR) RMSE: 7.527587408906e-12
Baseline (LR) MAE: 5.774439071210237e-12
RF RMSE: 96.32553394208085
RF MAE: 59.15137369635231
XGB RMSE: 70.97879786312072
XGB MAE: 41.228782653808594


#### 테스트 결과, 현재 데이터 구조에서는 XGBoost가 적합하다고 판단 -> XGBoost로 고정

## [2단계] 테스트 기간(7, 30, 60, 90일)별 성능 비교

In [ ]:
# 임의로 특정 교차로 선택
target = "2공단3"
df = df[df["intersection"] == target].copy()


df["roll3"] = (   # 3시간 이동 평균
df.groupby("intersection")["traffic_volume"]
      .rolling(3).mean()
      .reset_index(level=0, drop=True)
)

df["roll6"] = (   # 6시간 이동 평균
    df.groupby("intersection")["traffic_volume"]
      .rolling(6).mean()
      .reset_index(level=0, drop=True)
)

drop_cols = ["traffic_volume", "intersection", "datetime", "date"]

# 비교할 테스트 기간 목록
test_days_list = [7, 30, 60, 90]    
results = []

for test_days in test_days_list:

    split_date = df["date"].max() - pd.Timedelta(days=test_days)   # 분기 기준 날짜 설정

    train = df[df["date"] <= split_date]    # 학습 데이터
    test  = df[df["date"] > split_date]     # 테스트 데이터

    X_train = train.drop(columns=drop_cols)   # 학습 입력 변수
    y_train = train["traffic_volume"]         # 학습 타겟 변수  

    X_test = test.drop(columns=drop_cols)     # 테스트 입력 변수  
    y_test = test["traffic_volume"]           # 테스트 타겟 변수  

    # XGBoost 모델로 실행
    xgb = XGBRegressor(         # XGBoost 모델 생성
        n_estimators=100,       # 트리 개수
        max_depth=6,            # 트리 깊이
        learning_rate=0.05,     # 학습률
        random_state=42,        
        subsample=0.8,          # 샘플 비율
        colsample_bytree=0.8,   # 특성 샘플 비율
        tree_method="hist"
    )

    xgb.fit(X_train, y_train)        # 모델 학습
    pred_xgb = xgb.predict(X_test)   # 테스트 예측

    rmse_xgb = (mean_squared_error(y_test, pred_xgb)) ** 0.5    # RMSE
    mae_xgb = mean_absolute_error(y_test, pred_xgb)             # MAE

    # 결과 저장
    results.append({
        "test_days": test_days,
        "XGB_RMSE": rmse_xgb,
        "XGB_MAE": mae_xgb
    })


# 결과 표 출력
results_df = pd.DataFrame(results)
print(results_df)

   test_days    XGB_RMSE     XGB_MAE
0          7  200.323254  114.959496
1         30  168.213882   96.413803
2         60  210.644936  118.021614
3         90  203.655116  114.838440


#### 테스트 결과, 테스트 기간은 30일이 가장 성능이 우수

## [3단계] 클러스터별 모델 예측

In [ ]:
df["lag1"]  = df.groupby("intersection")["traffic_volume"].shift(1)   # 1시간 전 교통량
df["lag2"]  = df.groupby("intersection")["traffic_volume"].shift(2)   # 2시간 전 교통량  
df["lag3"]  = df.groupby("intersection")["traffic_volume"].shift(3)   # 3시간 전 교통량  

df["lag24"] = df.groupby("intersection")["traffic_volume"].shift(24)    # 24시간 전 교통량
df["lag48"] = df.groupby("intersection")["traffic_volume"].shift(48)    # 48시간 전 교통량
df['lag72'] = df.groupby("intersection")['traffic_volume'].shift(72)    # 72시간 전 교통량

df["roll3"]  = (    # 3시간 이동 평균
    df.groupby("intersection")["traffic_volume"]
    .rolling(3).mean()
    .reset_index(0,drop=True)
)
df["roll6"]  = (    # 6시간 이동 평균
    df.groupby("intersection")["traffic_volume"]
    .rolling(6).mean()
    .reset_index(0,drop=True)
)
df["roll24"] = (    # 24시간 이동 평균
    df.groupby("intersection")["traffic_volume"]
    .rolling(24).mean()
    .reset_index(0,drop=True)
)
df['roll48'] = (    # 48시간 이동 평균
    df.groupby("intersection")['traffic_volume']
    .rolling(48).mean()
    .reset_index(level=0, drop=True)
)

df = df.dropna() 


# 시간 파생변수
df["dayofweek"] = df["datetime"].dt.dayofweek

# Cyclic Encoding
df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24)      # 시간 사인 변환
df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24)      # 시간 코사인 변환

df["dow_sin"] = np.sin(2*np.pi*df["dayofweek"]/7)   # 요일 사인 변환
df["dow_cos"] = np.cos(2*np.pi*df["dayofweek"]/7)   # 요일 코사인 변환


# 클러스터별로 분리
df_c0 = df[df["cluster"] == 0].copy()   # 중/저혼잡형
df_c1 = df[df["cluster"] == 1].copy()   # 상시혼잡형

print("Cluster 0 데이터 수:", len(df_c0))
print("Cluster 1 데이터 수:", len(df_c1))


# Train/Test 분리 함수
def split_data(df, test_days=30):
    split_date = df["date"].max() - pd.Timedelta(days=test_days)
    
    train = df[df["date"] <= split_date]
    test  = df[df["date"] > split_date]

    drop_cols = ["traffic_volume", "intersection", "datetime", "date"]

    X_train = train.drop(columns=drop_cols)
    y_train = train["traffic_volume"]

    X_test = test.drop(columns=drop_cols)
    y_test = test["traffic_volume"]

    return X_train, y_train, X_test, y_test


# 모델 학습 함수
def train_xgb(X_train, y_train):
    model = XGBRegressor(
        n_estimators=400,
        max_depth=10,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=1.0,
        gamma=1,
        min_child_weight=2,
        random_state=42
    )
    model.fit(X_train, y_train)
    return model

# 모델 평가 함수
def evaluate(model, X_test, y_test):
    pred = model.predict(X_test)
    rmse = (mean_squared_error(y_test, pred))**0.5
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)
    return rmse, mae, r2


# 클러스터별 예측 실행
# --- Cluster 0 ---
X_train0, y_train0, X_test0, y_test0 = split_data(df_c0)
model0 = train_xgb(X_train0, y_train0)
rmse0, mae0, r20 = evaluate(model0, X_test0, y_test0)

# --- Cluster 1 ---
X_train1, y_train1, X_test1, y_test1 = split_data(df_c1)
model1 = train_xgb(X_train1, y_train1)
rmse1, mae1, r21 = evaluate(model1, X_test1, y_test1)

print("\n==============================")
print("📌 클러스터별 예측 성능 (최종)")
print("==============================")

print(f"[Cluster 0 - 중·저혼잡형]  RMSE: {rmse0:.2f} | MAE: {mae0:.2f} | R²: {r20:.4f}")
print(f"[Cluster 1 - 상시혼잡형]  RMSE: {rmse1:.2f} | MAE: {mae1:.2f} | R²: {r21:.4f}")

print("\n🎯 클러스터 기반 예측 완료!")


Cluster 0 데이터 수: 333426
Cluster 1 데이터 수: 251502

📌 클러스터별 예측 성능 (최종)
[Cluster 0 - 중·저혼잡형]  RMSE: 41.54 | MAE: 20.06 | R²: 0.9984
[Cluster 1 - 상시혼잡형]  RMSE: 67.90 | MAE: 34.59 | R²: 0.9983

🎯 클러스터 기반 예측 완료!
